In [1]:
import pandas as pd
from google.cloud import bigquery
import pandas as pd
import numpy as np
from google.cloud import bigquery
from bq_helper import BigQueryHelper
import math

In [14]:
# Set the path of your credentials file (alternatively you can do this in your system path but I haven't had success)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/keaton/carter/My Project-a95df9756ff9.json'

bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")

STATES = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii',
'Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota',
'Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina',
'North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee',
'Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming'] 

STATEABVS = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

FEATURES = ['o3','co','so2','no2','pm25_frm', 'pressure', 'temperature', 'wind']

SAMPLE_DURATIONS = ['8-HR RUN AVG BEGIN HOUR','8-HR RUN AVG END HOUR', '3-HR BLK AVG', '1 HOUR', '24 HOUR', '1 HOUR', '1 HOUR', '1 HOUR']

In [3]:
# Helper method for itterating through querries
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [4]:
# Helper method for ensuring all dataframes have the same index
def fillIndex(data):

    X = data.copy()

    temp = []
    for i in range(50):
        temp.append(np.nan)

    for year in range(1990,2020):
        if not (year in X.index):
            X.loc[year] = temp

    X = X.sort_index(ascending = True)

    return X

In [5]:
# Helper method for combining dataframes
def unifyDataFrames(df1, df2, df3):
    df1copy = np.array(fillIndex(df1))
    df2copy = np.array(fillIndex(df2))
    df3copy = np.array(fillIndex(df3))

    for i in range(len(df1copy)-1):
        for j in range(len(df1copy[0])-1):
            if math.isnan(df1copy[i][j]):
                if not(math.isnan(df2copy[i][j])):
                       df1copy[i][j] = df2copy[i][j]
                elif not(math.isnan(df3copy[i][j])):
                        df1copy[i][j] = df3copy[i][j]

    return pd.DataFrame(df1copy)

In [6]:
# Returns a dataframe with the average measurement of a feature per year for each state
def averagePollutants(featureList, stateList, poc, sample_durations):
    QUERY = """
    SELECT EXTRACT(YEAR FROM feature.date_local) as Year , AVG(feature.arithmetic_mean) as STitle
    FROM
      `bigquery-public-data.epa_historical_air_quality.feature_daily_summary` as feature
      WHERE feature.poc = NUM AND  feature.state_name = 'State' AND feature.sample_duration = 'LENGTH'
    GROUP BY Year
    ORDER BY Year ASC
        """
    dict_pol={}
    counter = 0
    for f in featureList : 
        dict_pol[f] = None 
        for s in stateList :
            dic = {"State": s, "feature": f, "STitle": s.replace(' ', '_'), "NUM":str(poc), "LENGTH": sample_durations[counter]}
            query = replace_all(QUERY, dic)
            temp = bq_assistant.query_to_pandas(query).set_index('Year')
            dict_pol[f] = pd.concat([dict_pol[f], temp], axis=1, sort=False)
        counter += 1
    
    return dict_pol

In [7]:
# Creates dataframes for the average feature per year in each state for different poc
averageFeatureDF1 = averagePollutants(FEATURES, STATES, 1, SAMPLE_DURATIONS)
averageFeatureDF2 = averagePollutants(FEATURES, STATES, 2, SAMPLE_DURATIONS)
averageFeatureDF3 = averagePollutants(FEATURES, STATES, 3, SAMPLE_DURATIONS)

In [53]:
# Combines average feature dataframes to fill in holes caused by using different poc
finalFeatureAverages = {}
FEATURES = ['o3','co','so2','no2','pm25_frm', 'pressure', 'temperature', 'wind']
for f in FEATURES:
    finalFeatureAverages[f] = None
    temp = unifyDataFrames(averageFeatureDF1[f], averageFeatureDF2[f], averageFeatureDF3[f])
    finalFeatureAverages[f] = pd.concat([finalFeatureAverages[f], temp], axis=1, sort=False)

# Add years to the features in a terrible way
test = finalFeatureAverages['o3']
yrs = []
for i in range(1990,2020):
    yrs.append([i]*50)

yrs_np = np.array(yrs)
finalFeatureAverages['year'] = pd.DataFrame(yrs_np)

# Add states to the features in an equally terrible way
sts = []
for i in range(1990,2020):
    sts.append(np.arange(50))

sts = np.array(sts)
finalFeatureAverages['state'] = pd.DataFrame(sts)

FEATURES = ['o3','co','so2','no2','pm25_frm', 'pressure', 'temperature', 'wind', 'year', 'state']

for f in FEATURES:
    filename = f +'.csv'
    finalFeatureAverages[f].to_csv(filename, index=False)

In [54]:
honeyData = pd.read_csv (r'honeyproduction.csv')

states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]


In [55]:
# Reads in honey data csv, formats data, adds honey to other features
processedHoneyData = []
for y in range(1990,1998):
    yearRow = []
    for s in states:
        yearRow.append(np.nan)
    processedHoneyData.append(yearRow)

for y in range(1998,2013):
    groupedYear = honeyData[honeyData['year']==y]
    groupedYear = groupedYear.loc[:,['state', 'yieldpercol']] 
    rawHoneyData = np.array(groupedYear).T
 
    yearRow = []
    for s in states:
        if s in rawHoneyData[0]:
            yearRow.append( (int(groupedYear[groupedYear['state']==s].yieldpercol)))
        else:
            yearRow.append(np.nan)
    processedHoneyData.append(yearRow)

for y in range(2013,2020):
    yearRow = []
    for s in states:
        yearRow.append(np.nan)
    processedHoneyData.append(yearRow)
processedHoneyData = pd.DataFrame(processedHoneyData)
finalFeatureAverages['honey'] = None
finalFeatureAverages['honey'] = pd.concat([finalFeatureAverages['honey'], processedHoneyData], axis=1, sort=False)

In [57]:
# Reformats data into feature vectors
rawData = []
for f in range(10):
    rawData.append(np.array(finalFeatureAverages[FEATURES[f]]))
rawData.append(np.array(finalFeatureAverages['honey']))
rawData = np.array(rawData)

featureVectors = []
for i in range(len(rawData[0])-1):
    for j in range(len(rawData[0][0])-1):
        if not (np.isnan(np.sum(rawData[:,i,j]))):
            featureVectors.append(rawData[:,i,j])
        
featureVectors = np.array(featureVectors)

pd.DataFrame(featureVectors).to_csv('completeFeatureVectors.csv', index = False)